In [ ]:
from azureml.core import Workspace, Datastore, Dataset
from azureml.core import Experiment
import pandas as pd
import os
from azureml.core.compute import ComputeTarget
from azureml.contrib.automl.pipeline.steps import AutoMLPipelineBuilder
from azureml.pipeline.core import Pipeline

In [ ]:
import shutil
import sys 
from scripts.helper import get_forecasting_output

In [ ]:
ws= Workspace.from_config()

In [ ]:
dstore = ws.get_default_datastore()

In [ ]:
experiment = Experiment(ws, 'manymodels-forecasting-pipeline-pandas-data')

In [ ]:
filedst_10_models = Dataset.get_by_name(ws, name='MMSA_Sample_inference')
filedst_10_models_input = filedst_10_models.as_named_input('MMSA_Sample_inference')

In [ ]:
compute = ComputeTarget(ws, 'compute-cluster')

In [ ]:
training_experiment_name = "manymodels-training-pipeline-pandas-data"
training_pipeline_run_id = "your-run-id"

In [ ]:
partition_column_names = ['Store']

In [ ]:
inference_steps = AutoMLPipelineBuilder.get_many_models_batch_inference_steps(experiment=experiment,
                                                                              inference_data=filedst_10_models_input,
                                                                              compute_target=compute,
                                                                              node_count=4,
                                                                              process_count_per_node=4,
                                                                              run_invocation_timeout=300,
                                                                              output_datastore=dstore,
                                                                              train_experiment_name=training_experiment_name,
                                                                              train_run_id=training_pipeline_run_id,
                                                                              partition_column_names=partition_column_names,
                                                                              time_column_name="Date",
                                                                              target_column_name="Sales")

In [ ]:
pipeline = Pipeline(workspace = ws, steps=inference_steps)
run = experiment.submit(pipeline)

In [ ]:
run.wait_for_completion(show_output=True)

In [ ]:
published_pipeline = pipeline.publish(name = 'automl_score_many_models_pandas',
                                      description = 'MMSA Solution using x data',
                                      version = '1',
                                      continue_on_step_failure = False)

In [ ]:
forecasting_results_name = "forecasting_results"
forecasting_output_name = "many_models_inference_output"

forecast_file = get_forecasting_output(run, forecasting_results_name, forecasting_output_name)
df = pd.read_csv(forecast_file, delimiter=" ", header=None)
df.columns = ["Date", "Store", "Sales", "Predicted" ]
print("Prediction has ", df.shape[0], " rows. Here the first 10.")
df.head(10)

In [ ]:
# from azureml.pipeline.core import Schedule, ScheduleRecurrence
    
# forecasting_pipeline_id = published_pipeline.id

# recurrence = ScheduleRecurrence(frequency="Month", interval=1, start_time="2020-01-01T09:00:00")
# recurring_schedule = Schedule.create(ws, name="automl_forecasting_recurring_schedule", 
#                             description="Schedule Forecasting Pipeline to run on the first day of every week",
#                             pipeline_id=forecasting_pipeline_id, 
#                             experiment_name=experiment.name, 
#                             recurrence=recurrence)